# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 48 new papers today
          24 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/23 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2302.03697


extracting tarball to tmp_2302.03697...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.03699


extracting tarball to tmp_2302.03699...

...

extracting tarball to tmp_2302.04239...

 done.


Retrieving document from  https://arxiv.org/e-print/2302.04258


extracting tarball to tmp_2302.04258...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.03699-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.03699) | **Quantifying the energetics of molecular superbubbles in PHANGS galaxies**  |
|| E. J. Watkins, et al. -- incl., <mark>K. Kreckel</mark>, <mark>E. Schinnerer</mark>, <mark>J. C. Lee</mark>, <mark>J. Li</mark> |
|*Appeared on*| *2023-02-09*|
|*Comments*| *20 pages, 14 figures, 3 tables. Submitted to A&A. Abstract abridged for arXiv*|
|**Abstract**| Star formation and stellar feedback are interlinked processes that redistribute energy and matter throughout galaxies. When young, massive stars form in spatially clustered environments, they create pockets of expanding gas termed superbubbles. As these processes play a critical role in shaping galaxy discs and regulating the baryon cycle, measuring the properties of superbubbles provides important input for galaxy evolution models. With wide coverage and high angular resolution ($\sim$50-150 pc) of the PHANGS-ALMA $^{12}$CO (2-1) survey, we can now resolve and identify a statistically representative number of superbubbles with molecular gas in nearby galaxies. We identify superbubbles by requiring spatial correspondence between shells in CO with stellar populations identified in PHANGS-HST, and combine the properties of the stellar populations with CO to constrain feedback models and quantify their energetics. We visually identify 325 cavities across 18 PHANGS-ALMA galaxies, 88 of which have clear superbubble signatures (unbroken shells, central clusters, kinematic signatures of expansion). We measure their radii and expansion velocities using CO to dynamically derive their ages and the mechanical power driving the bubbles, which we use to compute the expected properties of the parent stellar populations driving the bubbles. We find consistency between the predicted and derived stellar ages and masses of the stellar populations if we use a supernova (SN) model that injects energy with a coupling efficiency of 5-12%, whereas wind models fail to explain stellar ages we measure. Not only does this confirm molecular gas accurately traces superbubble properties, but it also provides key observational constraints for superbubble models. We also find evidence that the bubbles are sweeping up gas as they expand and speculate that these sites have the potential to host new generations of stars. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.04138-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.04138) | **EDEN Survey: Small Transiting Planet Detection Limits and Constraints on  the Occurrence Rates for Late M Dwarfs within 15 pc**  |
|| Jeremy Dietrich, et al. -- incl., <mark>Nicolas Kurtovic</mark>, <mark>Thomas Henning</mark>, <mark>Samantha Brown-Sevilla</mark>, <mark>Remo Burn</mark>, <mark>Timmy N. Delage</mark>, <mark>Lizxandra Flores-Rivera</mark>, <mark>Riccardo Franceschi</mark>, <mark>Sofia Savvidou</mark>, <mark>Jonas Syed</mark>, <mark>An-Li Tsai</mark>, <mark>Chia-Lung Lin</mark>, <mark>Chi-Sheng Lin</mark>, <mark>Hung-Chin Lin</mark> |
|*Appeared on*| *2023-02-09*|
|*Comments*| *27 pages, 11 figures*|
|**Abstract**| Earth-sized exoplanets that transit nearby, late spectral type red dwarfs will be prime targets for atmospheric characterization in the coming decade. Such systems, however, are difficult to find via wide-field transit surveys like Kepler or TESS. Consequently, the presence of such transiting planets is unexplored and the occurrence rates of short-period Earth-sized planets around late M dwarfs remain poorly constrained. Here, we present the deepest photometric monitoring campaign of 22 nearby late M dwarf stars, using data from over 500 nights on seven 1-2 meter class telescopes. Our survey includes all known single quiescent northern late M dwarfs within 15 pc. We use transit-injection-and-recovery tests to quantify the completeness of our survey, successfully identify most ($>80\%$) transiting short-period (0.5-1 d) super-Earths ($R > 1.9 R_\oplus$), and are sensitive ($\sim50\%$) to transiting Earth-sized planets ($1.0-1.2 R_\oplus$). Our high sensitivity to transits with a near-zero false positive rate demonstrates an efficient survey strategy. Our survey does not yield a transiting planet detection, yet it provides the most sensitive upper limits on transiting planets orbiting our target stars. Finally, we explore multiple hypotheses about the occurrence rates of short-period planets (from Earth-sized planets to giant planets) around late M dwarfs. We show, for example, that giant planets at short periods ($<1$ day) are uncommon around our target stars. Our dataset provides some insight into occurrence rates of short-period planets around TRAPPIST-1-like stars, and our results can help test planetary formation and system evolution models, as well as guide future observations of nearby late M dwarfs. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.04239-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.04239) | **Simulation of CH$_3$OH ice UV photolysis under laboratory conditions**  |
|| W. R. M. Rocha, et al. -- incl., <mark>G. Perotti</mark> |
|*Appeared on*| *2023-02-09*|
|*Comments*| *Accepted for publication in A&A on 08-Feb-2023*|
|**Abstract**| Methanol is the most complex molecule securely identified in interstellar ices and is a key chemical species for understanding chemical complexity in astrophysical environments. Important aspects of the methanol ice photochemistry are still unclear such as the branching ratios and photo-dissociation cross-sections at different temperatures and irradiation fluxes. This work aims at a quantitative agreement between laboratory experiments and astrochemical modelling of the CH3OH ice UV photolysis. This work also allows us to better understand which processes govern the methanol ice photochemistry present in laboratory experiments. We use ProDiMo to simulate the conditions of laboratory measurements. The simulations start with simple chemistry consisting only of methanol ice and helium to mimic the residual gas in the experimental chamber. A surface chemical network enlarged by photo-dissociation reactions is used to study the chemical reactions within the ice. Additionally, different surface chemistry parameters (surface competition, tunnelling, thermal diffusion and reactive desorption) are adopted to check those that reproduce the experimental results. The chemical models with ProDiMo can reproduce the methanol ice destruction via UV photodissociation at temperatures of 20, 30, 50 and 70 K as observed in the experiments. We note that the results are sensitive to different branching ratios after photolysis and to the mechanisms of reactive desorption. In the simulations of a molecular cloud at 20 K, we observed an increase in the methanol gas abundance of one order of magnitude, with a similar decrease in the solid-phase abundance. Comprehensive astrochemical models provide new insights into laboratory experiments as the quantitative understanding of the processes that govern the reactions within the ice. Ultimately, those insights can help to better interpret astronomical observations. |

## Failed papers

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.04194-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.04194) | **Modeling the Galaxy Distribution in Clusters using Halo Cores**  |
|| Danila Korytov, et al. -- incl., <mark>Lindsey Bleem</mark>, <mark>Katrin Heitmann</mark>, <mark>Joseph Hollowed</mark> |
|*Appeared on*| *2023-02-09*|
|*Comments*| *17 pages, 19 figures, 1 Appendix*|
|**Abstract**| The galaxy distribution in dark matter-dominated halos is expected to approximately trace the details of the underlying dark matter substructure. In this paper we introduce halo `core-tracking' as a way to efficiently follow the small-scale substructure in cosmological simulations and apply the technique to model the galaxy distribution in observed clusters. The method relies on explicitly tracking the set of particles identified as belonging to a halo's central density core, once a halo has attained a certain threshold mass. The halo cores are then followed throughout the entire evolution of the simulation. The aim of core-tracking is to simplify substructure analysis tasks by avoiding the use of subhalos and, at the same time, to more easily account for the so-called ``orphan'' galaxies, which have lost substantial dark mass due to tidal stripping. We show that simple models based on halo cores can reproduce the number and spatial distribution of galaxies found in optically-selected clusters in the Sloan Digital Sky Survey. We also discuss future applications of the core-tracking methodology in studying the galaxy-halo connection. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.04258-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.04258) | **Estimating the longitudinal magnetic field in the chromosphere of  quiet-Sun magnetic concentrations**  |
|| S. Esteban Pozuelo, et al. -- incl., <mark>A. Asensio Ramos</mark> |
|*Appeared on*| *2023-02-09*|
|*Comments*| *14 pages, 14 figures. Accepted for publication in Astronomy & Astrophysics; abstract has been abridged*|
|**Abstract**| Details of the magnetic field in the quiet Sun chromosphere are key to our understanding of essential aspects of the solar atmosphere. We aim to determine the longitudinal magnetic field component (B_lon) of quiet Sun regions depending on their size. We estimated B_lon by applying the weak-field approximation (WFA) to high-spatial-resolution Ca II 854.2 nm data taken with the Swedish 1m Solar Telescope. Specifically, we analyzed the estimates inferred for different spectral ranges using the data at the original cadence and temporally integrated signals. The longitudinal magnetic field in each considered plasma structure correlates with its size. Using a spectral range restricted to the line core leads to chromospheric longitudinal fields varying from 50 G at the edges to 150-500 G at the center of the structure. These values increase as the spectral range widens due to the photospheric contribution. However, the difference between this contribution and the chromospheric one is not uniform for all structures. Small and medium-sized concentrations show a steeper height gradient in B_lon compared to their chromospheric values, so estimates for wider ranges are less trustworthy. Signal addition does not alleviate this situation as the height gradients in B_lon are consistent with time. Finally, despite the amplified noise levels that deconvolving processes may cause, data restored with the destretching technique show similar results, though are affected by smearing. We obtained B_lon estimates similar to those previously found, except for large concentrations and wide spectral ranges. In addition, we report a correlation between the height variation of B_lon compared to the chromospheric estimates and the concentration size. This correlation affects the difference between the photospheric and chromospheric magnetic flux values and the reliability of the estimates for wider spectral ranges. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.03721-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.03721) | **Radiation shielding of protoplanetary discs in young star-forming  regions**  |
|| Martijn J. C. Wilhelm, et al. -- incl., <mark>Sean C. Lewis</mark> |
|*Appeared on*| *2023-02-09*|
|*Comments*| *23 pages, 22 figures, 1 table, accepted for publication in MNRAS*|
|**Abstract**| Protoplanetary discs spend their lives in the dense environment of a star forming region. While there, they can be affected by nearby stars through external photoevaporation and dynamic truncations. We present simulations that use the AMUSE framework to couple the Torch model for star cluster formation from a molecular cloud with a model for the evolution of protoplanetary discs under these two environmental processes. We compare simulations with and without extinction of photoevaporation-driving radiation. We find that the majority of discs in our simulations are considerably shielded from photoevaporation-driving radiation for at least 0.5 Myr after the formation of the first massive stars. Radiation shielding increases disc lifetimes by an order of magnitude and can let a disc retain more solid material for planet formation. The reduction in external photoevaporation leaves discs larger and more easily dynamically truncated, although external photoevaporation remains the dominant mass loss process. Finally, we find that the correlation between disc mass and projected distance to the most massive nearby star (often interpreted as a sign of external photoevaporation) can be erased by the presence of less massive stars that dominate their local radiation field. Overall, we find that the presence and dynamics of gas in embedded clusters with massive stars is important for the evolution of protoplanetary discs. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.03871-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.03871) | **A Three-Dimensional Analytical Model of the Interstellar Extinction  within the Nearest Kiloparsec**  |
|| G. A. Gontcharov, et al. -- incl., <mark>N. Hebdon</mark> |
|*Appeared on*| *2023-02-09*|
|*Comments*| *Published in Astronomy Letters, 2022, 48, 578-600*|
|**Abstract**| We present a new version of our analytical model of the spatial interstellar extinction variations within the nearest kiloparsec. This model treats the 3D dust distribution as a superposition of three overlapping layers: (1) the layer along the Galactic midplane, (2) the layer in the Gould Belt, and (3) the layer passing through the Cepheus and Chamaeleon dust cloud complexes. In each layer the dust density decreases exponentially with increasing distance from the midplane of the layer. In addition, there are sinusoidal longitudinal extinction variations along the midplane of each layer. We have found the most probable values of 29 parameters of our model using four data sets: the 3D stellar reddening maps by Gontcharov (2017), Lallement et al. (2019), and Green et al. (2019) and the extinctions inferred by Anders et al. (2022) for 993291 giants from the Gaia EDR3. All of the data give similar estimates of the model parameters. The extinction for a star or a point in space is predicted by our model with an accuracy from $\sigma(A_\mathrm{V})=0.07$ to 0.37 for high and low Galactic latitudes, respectively. The natural fluctuations of the dust medium dominate in these values. When ignoring the fluctuations of the medium, the average extinction for an extended object (a galaxy, a star cluster, a dust cloud) or a small region of space is predicted by our model with an accuracy from $\sigma(A_\mathrm{V})=0.04$ to 0.15 for high and low Galactic latitudes, respectively. Green et al. (2019) and Anders et al. (2022) give in unison an estimate of $A_\mathrm{V}=0.12^m$ for the extinction at high latitudes across the whole Galactic dust half-layer above or below the Sun with the natural fluctuations of the medium $\sigma(A_\mathrm{V})=0.06^m$. Our model is a step to explain how a substantial amount of dust ended up far from the Galactic midplane. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'utf-8' codec can't decode byte 0xe3 in position 993: invalid continuation byte</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.03713-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.03713) | **VLA Legacy Survey of Molecular Gas in Massive Star-forming Galaxies at  High Redshift**  |
|| Marta Frias Castillo, et al. -- incl., <mark>Cheng-Lin Liao</mark>, <mark>Fabian Walter</mark>, <mark>Helmut Dannerbauer</mark> |
|*Appeared on*| *2023-02-09*|
|*Comments*| *10 pages, 15 figures. Accepted for publication in ApJ*|
|**Abstract**| We present initial results of an ongoing survey with the Karl G. Jansky Very Large Array targeting the CO($J$ = 1-0) transition in a sample of 30 submillimeter-selected, dusty star-forming galaxies at $z =$ 2-5 with existing mid--$J$ CO detections from ALMA and NOEMA, of which 17 have been fully observed. We detect CO(1-0) emission in 11 targets, along with three tentative ($\sim$1.5-2$\sigma$) detections; three galaxies are undetected. Our results yield total molecular gas masses of 6-23$\times$10$^{10}$ ($\alpha_\mathrm{CO}$/1) M$_\odot$, with gas mass fractions, $f_\mathrm{gas}$=$M_\mathrm{mol}$/($M_*$+$M_\mathrm{mol}$), of 0.1-0.8 and a median depletion time of (140$\pm$70) Myr. We find median CO excitation ratios of $r_{31}$ = 0.75$\pm$0.39 and $r_{41}$ = 0.63$\pm$0.44, with a significant scatter. We find no significant correlation between the excitation ratio and a number of key parameters such as redshift, CO(1-0) line width or $\Sigma_\mathrm{SFR}$. We only find a tentative positive correlation between $r_{41}$ and the star-forming efficiency, but we are limited by our small sample size. Finally, we compare our results to predictions from the SHARK semi-analytical model, finding a good agreement between the molecular gas masses, depletion times and gas fractions of our sources and their SHARK counterparts. Our results highlight the heterogeneous nature of the most massive star-forming galaxies at high-redshift, and the importance of CO(1--0) observations to robustly constrain their total molecular gas content and ISM properties. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure Lprime_v_z.pdf</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2302.03699.md
    + _build/html/tmp_2302.03699/./used_plots/method_illistration_2.png
    + _build/html/tmp_2302.03699/./used_plots/measured_bub_props-v1p3-peer-avg_velo.png
    + _build/html/tmp_2302.03699/./used_plots/average_tpeak_intensity-all_norm-err-peer_reviewed.png
exported in  _build/html/2302.04138.md
    + _build/html/tmp_2302.04138/./TOIs_EDENSensitivity-01.png
    + _build/html/tmp_2302.04138/./LOT_20201118_J0419.png
    + _build/html/tmp_2302.04138/./KUIPER_20201118_J0419.png
    + _build/html/tmp_2302.04138/./VATT_20201222_J0419.png
    + _build/html/tmp_2302.04138/./average_sensitivity_map.png
    + _build/html/tmp_2302.04138/./average_detection_map.png
exported in  _build/html/2302.04239.md
    + _build/html/tmp_2302.04239/./Figures/M1_50K.png
    + _build/html/tmp_2302.04239/./Figures/M3_30K.png
    + _build/html/tmp_2302.04239/./Figures/CH2OH_Formations_row_V2.png
    + _build/html/tmp_2302.04239/./Figures/CO_Formations_row_V2.png
    + _bui

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\SII}{[S\textsc{ii}]}$
$\newcommand{\OI}{[O\textsc{i}]}$
$\newcommand{\OIII}{[O\textsc{iii}]}$
$\newcommand{\NII}{[N\textsc{ii}]}$
$\newcommand{\kms}{km~s^{-1}}$
$\newcommand{\h}{H_2}$
$\newcommand{\msun}{M_\odot}$
$\newcommand{\e}{\times10}$
$\newcommand{\HII}{H\textsc{ii}}$
$\newcommand{\HI}{H\textsc{i}}$
$\newcommand{\Ha}{H\alpha}$
$\newcommand{\Hb}{H\beta}$
$\newcommand{\CO}{^{12}CO}$
$\newcommand{\Tpeak}{T_\text{peak}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\SII$}{[S$\textsc${ii}]}$
$\newcommand{$\OI$}{[O$\textsc${i}]}$
$\newcommand{$\OI$II}{[O$\textsc${iii}]}$
$\newcommand{$\NII$}{[N$\textsc${ii}]}$
$\newcommand{$\kms$}{km~s^{-1}}$
$\newcommand{$\h$}{H_2}$
$\newcommand{$\msun$}{M_\odot}$
$\newcommand{$\e$}{\times10}$
$\newcommand{$\HII$}{H$\textsc${ii}}$
$\newcommand{$\HI$}{H$\textsc${i}}$
$\newcommand{$\Ha$}{H\alpha}$
$\newcommand{$\Hb$}{H\beta}$
$\newcommand{$\CO$}{^{12}CO}$
$\newcommand{$\Tpeak$}{T_\text{peak}}$</div>



<div id="title">

# Quantifying the energetics of molecular superbubbles in PHANGS galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.03699-b31b1b.svg)](https://arxiv.org/abs/2302.03699)<mark>Appeared on: 2023-02-09</mark> - _20 pages, 14 figures, 3 tables. Submitted to A&A. Abstract abridged for arXiv_

</div>
<div id="authors">

E. J. Watkins, et al. -- incl., <mark>K. Kreckel</mark>, <mark>E. Schinnerer</mark>, <mark>J. C. Lee</mark>, <mark>J. Li</mark>

</div>
<div id="abstract">

**Abstract:** Star formation and stellar feedback are interlinked processes that redistribute energy, turbulence and material throughout galaxies. Because young, and massive stars form in spatially clustered environments, they create pockets of expanding gas termed superbubbles, which retain information about the physical processes that drive them. As these processes play a critical role in shaping galaxy discs and regulating the baryon cycle, measuring the properties of superbubbles provides important input for galaxy evolution models.With wide coverage and high angular resolution ($\sim$50--150 pc) of the PHANGS--ALMA$\CO$({\it J}=2$-$1) survey, we can now resolve, identify and characterise statistically representative numbers of superbubbles with molecular gas in nearby galaxies.We identify superbubbles by requiring spatial correspondence between shells in CO with stellar populations identified in PHANGS--HST, and combine the properties of the stellar populations with CO to constrain feedback models and quantify their energetics. We visually identify 325 cavities across 18 PHANGS--ALMA galaxies, 88 of which have clear superbubble signatures (unbroken shells, central clusters, kinematic signatures of expansion). We measure their radii and expansion velocities using CO (2--1) to dynamically derive their ages and the mechanical power driving the bubbles, which we use to compute the expected properties of the parent stellar populations driving the bubbles.We find consistency between the predicted and derived stellar ages and masses of the stellar populations if we use a supernova (SN) model that injects energy with a coupling efficiency of 5--12\%, whereas wind models fail to explain stellar ages we measure. Not only does this confirm molecular gas accurately traces superbubble properties, but it also provides key observational constraints for superbubble models. We also find evidence that the bubbles are sweeping up gas as they expand and speculate that these sites have the potential to host new generations of stars.This work demonstrates that molecular superbubbles provide  novel quantitative constraints on SNe feedback efficiencies and gas clearing times, and represent a promising environment to search for the propagation of star formation, all of which are needed to understand what sets the observed star formation rates in galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2302.03699/./used_plots/method_illistration_2.png" alt="Fig10" width="100%"/>

**Figure 10. -** Illustrating bubble identification and elimination criteria in section of NGC 1566. \textbf{1.} CO $\Tpeak$(red), $\Ha$(green), and HST \textit{B}-band (blue) combined into a false colour image at their original resolution to identify superbubbles using multi-wavelength information. \textbf{2.} Manually fitting radii and their centres using the CO $\Tpeak$ map. Cyan and dashed white circles show catalogued bubbles that were analysed or ignored respectively. The blue box outlines the bubble examined in all remaining panels (Bubble 36 in Table \ref{tab:props}). \textbf{3.} Checking three channel maps in CO to confirm if bubble emission is significant in multiple consecutive velocity bands. If not, the bubble is removed from the sample. \textbf{4.} Horizontal and vertical PV diagrams to confirm expansion signature is present. Grey ellipse shows present expansion signature. If unable to confirm, bubble is removed. \textbf{5.} Illustrating average spectra around the bubble. If identifiable background or foreground emission is found, bubble is removed. All spectra shown here are free of contaminating emission. (*fig:method*)

</div>
<div id="div_fig2">

<img src="tmp_2302.03699/./used_plots/measured_bub_props-v1p3-peer-avg_velo.png" alt="Fig12" width="100%"/>

**Figure 12. -** Histogram distributions of the bubble radii \textit{(left)}, expansion velocity \textit{(middle)} and shell mass \textit{(right)} for all 88 superbubbles. The dashed red line in the left panel shows a power-law fit to the radii with an exponent of $-3.2$. Thick vertical grey and black lines show the mean and median of each distribution, respectively, and coloured filled regions show the 16\textsuperscript{th}--84\textsuperscript{th} sigma percentiles. (*fig:meas-bub_props*)

</div>
<div id="div_fig3">

<img src="tmp_2302.03699/./used_plots/average_tpeak_intensity-all_norm-err-peer_reviewed.png" alt="Fig1" width="100%"/>

**Figure 1. -** Averaged normalised intensity profile of bubbles categorised as \textit{closed rings}. Solid red line shows the mean profile, dashed black line shows the median profile and filled grey region shows the statistical spread of the data measured using the standard deviation ($\sigma$). (*fig:profile*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.04138-b31b1b.svg)](https://arxiv.org/abs/2302.04138)<mark>Appeared on: 2023-02-09</mark> - _27 pages, 11 figures_

</div>
<div id="authors">

Jeremy Dietrich, et al. -- incl., <mark>Nicolas Kurtovic</mark>, <mark>Thomas Henning</mark>, <mark>Samantha Brown-Sevilla</mark>, <mark>Remo Burn</mark>, <mark>Timmy N. Delage</mark>, <mark>Lizxandra Flores-Rivera</mark>, <mark>Riccardo Franceschi</mark>, <mark>Sofia Savvidou</mark>, <mark>Jonas Syed</mark>, <mark>An-Li Tsai</mark>, <mark>Chia-Lung Lin</mark>, <mark>Chi-Sheng Lin</mark>, <mark>Hung-Chin Lin</mark>

</div>
<div id="abstract">

**Abstract:** Earth-sized exoplanets that transit nearby, late spectral type red dwarfs will be prime targets for atmospheric characterization in the coming decade. Such systems, however, are difficult to find via wide-field transit surveys like\textit{Kepler}or TESS. Consequently, the presence of such transiting planets is unexplored and the occurrence rates of short-period Earth-sized planets around late M dwarfs remain poorly constrained. Here, we present the deepest photometric monitoring campaign of\ntargetsnearby late M dwarf stars, using data from over 500 nights on seven 1-2 meter class telescopes. Our survey includes all known single quiescent northern late M dwarfs within 15 pc.  We use transit-injection-and-recovery tests to quantify the completeness of our survey, successfully identify most ($>80\%$) transiting short-period (0.5-1 d) super-Earths (R$>1.9 R_\oplus$), and are sensitive ($\sim$50\%) to transiting Earth-sized planets (1.0--1.2$R_\oplus$). Our high sensitivity to transits with a near-zero false positive rate demonstrates an efficient survey strategy. Our survey does not yield a transiting planet detection, yet it provides the most sensitive upper limits on transiting planets orbiting our target stars. Finally, we explore multiple hypotheses about the occurrence rates of short-period planets (from Earth-sized planets to giant planets) around late M dwarfs. We show, for example, that giant planets at short periods ($<$1 day) are uncommon around our target stars. Our dataset provides some insight into occurrence rates of short-period planets around TRAPPIST-1-like stars, and our results can help test planetary formation and system evolution models, as well as guide future observations of nearby late M dwarfs.

</div>

<div id="div_fig1">

<img src="tmp_2302.04138/./TOIs_EDENSensitivity-01.png" alt="Fig5" width="100%"/>

**Figure 5. -** TESS objects of interest (TOIs) by stellar effective temperature and planet candidate radius. The temperature and planet radius distributions of the TOIs show a marked drop toward late spectral types ($>$M4--M6) and for small planets ($R_p<2$ $R_\oplus$). The green shaded region represents the sensitivity of our EDEN survey, which is complementary to that of TESS. (*fig:TESSTOIST*)

</div>
<div id="div_fig2">

<img src="tmp_2302.04138/./LOT_20201118_J0419.png" alt="Fig9.1" width="33%"/><img src="tmp_2302.04138/./KUIPER_20201118_J0419.png" alt="Fig9.2" width="33%"/><img src="tmp_2302.04138/./VATT_20201222_J0419.png" alt="Fig9.3" width="33%"/>

**Figure 9. -** The first look at the non-detrended light curves from a simple reference star calibration of each potential transit event, before full detrending is done. The transit-like features are marked with green lines and the name of the Project EDEN member who analyzed the quick look data. (*fig:EIC9FL*)

</div>
<div id="div_fig3">

<img src="tmp_2302.04138/./average_sensitivity_map.png" alt="Fig7.1" width="50%"/><img src="tmp_2302.04138/./average_detection_map.png" alt="Fig7.2" width="50%"/>

**Figure 7. -** The average sensitivity (top) and full detection probability including geometric transit likelihood (bottom) of the EDEN survey for planets with periods between 0.5 and 10 days and sizes between 0.6 and 3.5 Earth radii.  For ultra-short-period planets ($<$ 1 day), we reach 50\% sensitivity for planets with radius $1.5 R_\oplus$ or larger.  Due to the low geometric transit probability, the overall completeness is much lower and drops sharply with the orbital period.  The inner edge of the habitable zone is defined using the \citet[][]{Kopparapu2013} model based on the temperature and insolation (assuming a stellar effective temperature of 2600 K and a planetary albedo of 0.3). (*fig:map1*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\theequation}{R\arabic{equation}}$
$\newcommand{\theequation}{R\arabic{equation}}$
$\newcommand{\theequation}{R\arabic{equation}}$
$\newcommand{\theequation}{R\arabic{equation}}$
$\newcommand{\theequation}{R\arabic{equation}}$
$\newcommand{\arraystretch}{1.}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\@}{currentlabel}$
$\newcommand{\@}{currentlabel}$
$\newcommand{\@}{currentlabel}$
$\newcommand{\@}{currentlabel}$
$\newcommand{\@}{currentlabel}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\arraystretch$}{1.5}$
$\newcommand{$\arraystretch$}{1.1}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.5}$
$\newcommand{$\theequation$}{R\arabic{equation}}$
$\newcommand{$\theequation$}{R\arabic{equation}}$
$\newcommand{$\theequation$}{R\arabic{equation}}$
$\newcommand{$\theequation$}{R\arabic{equation}}$
$\newcommand{$\theequation$}{R\arabic{equation}}$
$\newcommand{$\arraystretch$}{1.}$
$\newcommand{$\arraystretch$}{1.2}$
$\newcommand{$\arraystretch$}{1.2}$
$\newcommand{$\arraystretch$}{1.2}$
$\newcommand{\@}{currentlabel}$
$\newcommand{\@}{currentlabel}$
$\newcommand{\@}{currentlabel}$
$\newcommand{\@}{currentlabel}$
$\newcommand{\@}{currentlabel}$</div>



<div id="title">

# OH ice UV photolysis under laboratory conditions

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.04239-b31b1b.svg)](https://arxiv.org/abs/2302.04239)<mark>Appeared on: 2023-02-09</mark> - _Accepted for publication in A&A on 08-Feb-2023_

</div>
<div id="authors">

W. R. M. Rocha, et al. -- incl., <mark><mark>G. Perotti</mark></mark>

</div>
<div id="abstract">

**Abstract:** Methanol is the most complex molecule securely identified in interstellar ices and is a key chemical species for understanding chemical complexity in astrophysical environments. Important aspects of the methanol ice photochemistry are still unclear such as the branching ratios and photo-dissociation cross-sections at different temperatures and irradiation fluxes.This work aims at a quantitative agreement between laboratory experiments and astrochemical modelling of the CH$_3$OH ice UV photolysis. Ultimately, this work allows us to better understand which processes govern the methanol ice photochemistry present in laboratory experiments.We use the\texttt{ProDiMo}code to simulate the radiation fields, pressures and pumping efficiencies characteristic of laboratory measurements. The simulations start with simple chemistry consisting only of methanol ice and helium to mimic the residual gas in the experimental chamber. A surface chemical network enlarged by photo-dissociation reactions is used to study the chemical reactions within the ice. Additionally, different surface chemistry parameters such as surface competition, tunnelling, thermal diffusion and reactive desorption are adopted to check those that reproduce the experimental results.The chemical models with the\texttt{ProDiMo}code including surface chemistry parameters can reproduce the methanol ice destruction via UV photodissociation at temperatures of 20, 30, 50 and 70 K as observed in the experiments. We also note that the results are sensitive to different branching ratios after photolysis and to the mechanisms of reactive desorption. In the simulations of a molecular cloud at 20 K, we observed an increase in the methanol gas abundance of one order of magnitude, with a similar decrease in the solid-phase abundance.Comprehensive astrochemical models provide new insights into laboratory experiments as the quantitative understanding of the processes that govern the reactions within the ice. Ultimately, those insights can help to better interpret astronomical observations.

</div>

<div id="div_fig1">

<img src="tmp_2302.04239/./Figures/M1_50K.png" alt="Fig18" width="100%"/>

**Figure 18. -** Grid of methanol ice photodissociation models at 50 K assuming different photodissociation cross-sections and BRs. The Model 1 in Table \ref{t7} is adopted in these plots. (*M1_50*)

</div>
<div id="div_fig2">

<img src="tmp_2302.04239/./Figures/M3_30K.png" alt="Fig13" width="100%"/>

**Figure 13. -** Grid of methanol ice photodissociation models at 30 K assuming different photodissociation cross-sections and BRs. The Model 3 in Table \ref{t7} is adopted in these plots. (*M3_30*)

</div>
<div id="div_fig3">

<img src="tmp_2302.04239/./Figures/CH2OH_Formations_row_V2.png" alt="Fig37.1" width="16%"/><img src="tmp_2302.04239/./Figures/CO_Formations_row_V2.png" alt="Fig37.2" width="16%"/><img src="tmp_2302.04239/./Figures/CO2_Formations_row_V2.png" alt="Fig37.3" width="16%"/><img src="tmp_2302.04239/./Figures/CH4_Formations_row_V2.png" alt="Fig37.4" width="16%"/><img src="tmp_2302.04239/./Figures/HCO_Formations_row_V3.png" alt="Fig37.5" width="16%"/><img src="tmp_2302.04239/./Figures/H2CO_Formations_row_V22.png" alt="Fig37.6" width="16%"/>

**Figure 37. -** Formation curves of selected daughter species (CH$_2$OH, CO, CO$_2$, CH$_4$, HCO and H$_2$CO) due to CH$_3$OH photolysis over fluence with respect to the initial methanol ice column density at four temperatures. The symbols are the experimental measurements and the solid lines are the results of ProDiMo simulations of the best photodissociation models (see Figure \ref{fits}). (*growth*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

198  publications in the last 7 days.
	 _build/html/2302.04239.md
	 _build/html/2302.04138.md
	 _build/html/2302.03699.md
	 _build/html/2302.03576.md
	 _build/html/2302.03042.md
	 _build/html/2302.03039.md
	 _build/html/2302.02890.md
	 _build/html/2302.02611.md
	 _build/html/2302.02429.md
	 _build/html/2302.01678.md
	 _build/html/2302.00450.md
	 _build/html/2301.13766.md
	 _build/html/2301.13526.md
	 _build/html/2301.12890.md
	 _build/html/2301.11942.md
	 _build/html/2301.10717.md
	 _build/html/2301.10666.md
	 _build/html/2301.10496.md
	 _build/html/2301.09621.md
	 _build/html/2301.09140.md
	 _build/html/2301.08770.md
	 _build/html/2301.08310.md
	 _build/html/2301.07990.md
	 _build/html/2301.07366.md
	 _build/html/2301.07199.md
	 _build/html/2301.07116.md
	 _build/html/2301.06873.md
	 _build/html/2301.06819.md
	 _build/html/2301.06575.md
	 _build/html/2301.05720.md
	 _build/html/2301.05718.md
	 _build/html/2301.05034.md
	 _build/html/2301.04656.md
	 _build/html/2301.04533.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers